In [162]:
import re
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.metrics.pairwise import cosine_similarity
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
data=pd.read_csv("/content/drive/My Drive/NLP/Project/Input/Datafiniti_Hotel_Reviews_Jun19.csv", encoding='utf8')


In [119]:
grouped_hotel=data.groupby("id")['reviews.text'].apply('.'.join).reset_index()
grouped_hotel.count()
grouped_hotel

,id,reviews.text
0,AV--lQdFRxPSIh2RmtbT,I read all the ratings before I booked the Bes...
1,AV--lmxKIxWefVJwsReI,This was our second visit to this beautiful re...
2,AV--luVA3-Khe5l_ZMEJ,"Unfortunately, the hurricane cancelled our tri..."
3,AV-AN728a4HuVbedGQnJ,We stayed at this hotel since the convention w...
4,AV-Cilg2a4HuVbedGQqK,We enjoyed our stay at the Hampton Inn in Brem...
...,...,...
1428,AWdgUTtXa4HuVbedaODu,We stayed for almost two weeks but at a condo ...
1429,AWdlE0rK3-Khe5l_tDlq,"A serviceable place to stay in Waikiki, not a ..."
1430,AWfPeemaa4HuVbeddb2K,Would not stay here again. We arrived at the h...
1431,AWfaJjQba4HuVbeddiSW,"Excellent property, great breakfast Buffett - ..."


In [163]:
sentences=word_tokenize(grouped_hotel["reviews.text"][1])
#sentences=[y for x in sentences for y in x ]
sentences

['This',
 'was',
 'our',
 'second',
 'visit',
 'to',
 'this',
 'beautiful',
 'resort',
 '.',
 'It',
 "'s",
 'island',
 'decor',
 ',',
 'accommodations',
 'and',
 'relaxed',
 'environment',
 'are',
 'wonderful',
 '.',
 'Beach',
 'views',
 'spectacular',
 '!',
 'Unfortunately',
 'our',
 'visit',
 'was',
 'cut',
 'short',
 'because',
 'of',
 'Hurricane',
 'Irma',
 'but',
 'management',
 'and',
 'staff',
 'support',
 'and',
 'consideration',
 'were',
 'truly',
 'appreciative',
 '.',
 'Will',
 'definitely',
 'return',
 '!']

In [165]:
clean_senteces=pd.Series(sentences).str.replace("[^a-zA-Z]"," ")

clean_senteces=[s.lower() for s in clean_senteces]

from nltk.corpus import stopwords
stop_words=stopwords.words('english')
def remove_stopwords(sen):
  sen_new=" ".join([i for i in sen if i not in stop_words])
  return sen_new

clean_senteces=[remove_stopwords(r.split())for r in clean_senteces]
#clean_senteces
cleanword=[]
for i in clean_senteces:
  if i!="":
    cleanword.append(i)
cleanword

['second',
 'visit',
 'beautiful',
 'resort',
 'island',
 'decor',
 'accommodations',
 'relaxed',
 'environment',
 'wonderful',
 'beach',
 'views',
 'spectacular',
 'unfortunately',
 'visit',
 'cut',
 'short',
 'hurricane',
 'irma',
 'management',
 'staff',
 'support',
 'consideration',
 'truly',
 'appreciative',
 'definitely',
 'return']

In [0]:
word_embeddings = {}
f= open("/content/drive/My Drive/NLP/Project/Input/wikiglove/glove.6B.300d.txt",encoding="utf-8")
for line in f:
  values=line.split()
  word=values[0]
  coefs=np.asarray(values[1:],dtype="float32")
  word_embeddings[word]=coefs
f.close()

In [125]:
len(word_embeddings)

400000

In [0]:
sentence_vector=[]

for i in cleanword:
  if len(i)!=0:
    v=sum([word_embeddings.get(w,np.zeros((300,)))for w in i.split()])/(len(i.split())+0.001)
    
  else:
    v=np.zeros((300,))
  sentence_vector.append(v)


In [167]:
len(sentences)

51

In [0]:
sim_mat=np.zeros([len(sentences),len(sentences)])

In [172]:
from sklearn.metrics.pairwise import cosine_similarity
for i in range (len(sentences)):
  for j in range(len(sentences)):
    if i!=j:
      sim_mat[i][j]=cosine_similarity(sentence_vector[i].reshape(1,300),sentence_vector[j].reshape(1,300))[0,0]
sim_mat

IndexError: ignored

In [0]:
import networkx as nx
nx_graph=nx.from_numpy_array(sim_mat)
score=nx.pagerank(nx_graph)

In [159]:
ranked_sentences=sorted(((score[i],s)for i,s in enumerate(sentences)),reverse=True)
for i in range(1):
  print(ranked_sentences[i][1])

This was our second visit to this beautiful resort.


In [160]:
score

{0: 0.2212068599172759,
 1: 0.20056970681735853,
 2: 0.18487635548581946,
 3: 0.20384552188874605,
 4: 0.1895015558908002}